In [ ]:
import os

#listar os arquivos na pasta sample_data no google collab
arquivos = os.listdir('/content/sample_data/')
print(arquivos)


['anscombe.json', 'README.md', 'arvores-tombadas.csv', 'arvorestombadas.geojson', 'arvorestombadas.geojsonmapa_interativo_arvores_tombadas.html', 'mnist_test.csv', 'mnist_train_small.csv', 'california_housing_train.csv', 'california_housing_test.csv']


In [ ]:
import folium
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# carregar o arquivo geojson das árvores tombadas
arquivo_geojson = '/content/sample_data/arvorestombadas.geojson'
arvores_geojson = gpd.read_file(arquivo_geojson)

# carregar o arquivo csv das árvores tombadas
arquivo_csv = '/content/sample_data/arvores-tombadas.csv'
arvores_csv = pd.read_csv(arquivo_csv, sep=';')  # assumindo que o csv usa ';' como delimitador

# garantir que o csv tenha latitude e longitude preenchidas para criar os pontos
arvores_csv['geometry'] = arvores_csv.apply(lambda row: Point(row['longitude'], row['latitude']) if pd.notnull(row['latitude']) and pd.notnull(row['longitude']) else None, axis=1)

# converter o csv para um geodataframe
arvores_csv_gdf = gpd.GeoDataFrame(arvores_csv, geometry='geometry', crs='EPSG:4326')

# remover as linhas com geometria inválida
arvores_csv_gdf = arvores_csv_gdf.dropna(subset=['geometry'])

# unir os dados do csv com o geodataframe do geojson com base na geometria (pontos de latitude e longitude)
arvores_combinadas = gpd.sjoin(arvores_geojson, arvores_csv_gdf, how='left', predicate='intersects')

# substituir nan em 'observacao' por um texto padrão
arvores_combinadas['observacao'] = arvores_combinadas['observacao'].fillna('sem observações')

# Definindo cores válidas do folium para mapear as cores personalizadas
cor_mapping = {
    'baobá': 'purple',
    'palmeira imperial': 'orange',
    'jaqueira': 'blue',
    'carolina': 'green',
    'gameleira': 'cyan',
    'barriguda': 'beige',
    'cajueiro': 'darkorange',
    'baóba': 'pink',
    'mangueira': 'darkviolet',
    'sapotizeiro': 'lightblue',
    'cajazeira': 'skyblue',
    'pau-de-jangada': 'rosybrown',
    'pau-brasil': 'darkgreen',
    'guapuruvu': 'gray',
    'pau mulato': 'lightgreen',
    'macaibeira': 'midnightblue'
}

# função para determinar a cor do ponto com base na espécie da árvore
def cor_ponto(row):
    especie = row['nmpopul'].strip().lower()
    return cor_mapping.get(especie, 'lightgray')

# função para ajustar a cor se houver uma observação
def cor_com_observacao(row):
    if row['observacao'] != 'sem observações':
        return 'red'
    else:
        return cor_ponto(row)

# contar quantas ocorrências de cada espécie
ocorrencias = arvores_combinadas['nmpopul'].value_counts()

# encontrar a árvore com maior e menor ocorrência
maior_ocorrencia = ocorrencias.idxmax()
menor_ocorrencia = ocorrencias.idxmin()

# criar o mapa base (centrado em recife)
mapa = folium.Map(location=[-8.05, -34.9], zoom_start=12)

#adicionar os pontos ao mapa com base nos dados combinados
for _, row in arvores_combinadas.iterrows():
    lat = row['geometry'].y
    lon = row['geometry'].x
    nome_popular = row['nmpopul']
    nome_cientifico = row['cdnmcient']
    endereco = row['nmender']
    observacao = row['observacao']
    familia = row['cdfam']

    # criar o ícone baseado na cor ajustada pela observação ou pela espécie
    icon = folium.Icon(color=cor_com_observacao(row), icon='fa-tree')

    # conteúdo do popup
    popup_content = f"""
        <b>Nome Popular:</b> {nome_popular}<br>
        <b>Nome Científico:</b> {nome_cientifico}<br>
        <b>Família:</b> {familia}<br>
        <b>Endereço:</b> {endereco}<br>
        <b>Observação:</b> {observacao}
    """

    # Criar pop-up com largura maior
    popup = folium.Popup(popup_content, max_width=500, min_width=300)  # Ajuste a largura conforme necessário

    # criar o marcador com popup e tooltip
    folium.Marker(
        location=[lat, lon],
        popup=popup,  # atribuindo o pop-up com a largura ajustada
        tooltip=nome_popular,
        icon=icon
    ).add_to(mapa)

# Adicionar a legenda com cores, espécies, quantidades e as árvores com maior e menor número de ocorrências
legend_html = f"""
<div style="position: fixed; bottom: 50px; left: 50px; width: 320px; height: 520px;
            background-color: white; border: 2px solid black; z-index: 9999; font-size: 14px; padding: 10px;">
    <b>Legenda das Árvores</b><br>
    <div><span style="background-color: purple; width: 15px; height: 15px; display: inline-block;"></span> Baobá</div>
    <div><span style="background-color: orange; width: 15px; height: 15px; display: inline-block;"></span> Palmeira Imperial</div>
    <div><span style="background-color: blue; width: 15px; height: 15px; display: inline-block;"></span> Jaqueira</div>
    <div><span style="background-color: green; width: 15px; height: 15px; display: inline-block;"></span> Carolina</div>
    <div><span style="background-color: cyan; width: 15px; height: 15px; display: inline-block;"></span> Gameleira</div>
    <div><span style="background-color: beige; width: 15px; height: 15px; display: inline-block;"></span> Barriguda</div>
    <div><span style="background-color: darkorange; width: 15px; height: 15px; display: inline-block;"></span> Cajueiro</div>
    <div><span style="background-color: pink; width: 15px; height: 15px; display: inline-block;"></span> Baóba</div>
    <div><span style="background-color: darkviolet; width: 15px; height: 15px; display: inline-block;"></span> Mangueira</div>
    <div><span style="background-color: lightblue; width: 15px; height: 15px; display: inline-block;"></span> Sapotizeiro</div>
    <div><span style="background-color: skyblue; width: 15px; height: 15px; display: inline-block;"></span> Cajazeira</div>
    <div><span style="background-color: rosybrown; width: 15px; height: 15px; display: inline-block;"></span> Pau-de-jangada</div>
    <div><span style="background-color: darkgreen; width: 15px; height: 15px; display: inline-block;"></span> Pau-Brasil</div>
    <div><span style="background-color: gray; width: 15px; height: 15px; display: inline-block;"></span> Guapuruvu</div>
    <div><span style="background-color: lightgreen; width: 15px; height: 15px; display: inline-block;"></span> Pau Mulato</div>
    <div><span style="background-color: midnightblue; width: 15px; height: 15px; display: inline-block;"></span> Macaibeira</div><br><br>

    <div style="border-top: 2px solid #ddd; padding-top: 15px; padding-bottom: 15px;">
        <b style="color: red;">Árvore com Maior Ocorrência:</b>
        <span style="font-weight: bold; color: green;">{maior_ocorrencia} - {ocorrencias[maior_ocorrencia]}</span><br>
        <b style="color: red;">Árvore com Menor Ocorrência:</b>
        <span style="font-weight: bold; color: blue;">{menor_ocorrencia} - {ocorrencias[menor_ocorrencia]}</span>
    </div>
</div>
"""
mapa.get_root().html.add_child(folium.Element(legend_html))

# salvar o mapa interativo em html
mapa.save('/content/sample_data/mapa_interativo_arvores_com_cores.html')

# exibir o mapa
mapa


<ipython-input-95-1ec0b09425c8>:82: UserWarning: color argument of Icon should be one of: {'lightred', 'beige', 'darkgreen', 'darkblue', 'black', 'purple', 'blue', 'gray', 'cadetblue', 'darkpurple', 'darkred', 'red', 'lightgreen', 'white', 'lightgray', 'pink', 'green', 'lightblue', 'orange'}.
  icon = folium.Icon(color=cor_com_observacao(row), icon='fa-tree')
